In [1]:
import os
import os.path as osp
import json
import pickle
import collections as C
import functools as F
import itertools as I
import random

import math_verify as M
from tqdm import tqdm

from common.utils import remove_comments

In [2]:
'/home/ma-user/local_cache/zhaoxlpku/PromptCoT-DS-Dataset/train.jsonl'
'/home/ma-user/local_cache/zhaoxlpku/PromptCoT-QwQ-Dataset/train.jsonl'
'/home/ma-user/local_cache/dyyyyyyyy/ScaleQuest-Math/train.json'

'/cache/data/MUSTARDSauce'

'/cache/data/MUSTARDSauce'

### PromptCoT-DS-Dataset

In [3]:
load_path = '/home/ma-user/local_cache/zhaoxlpku/PromptCoT-DS-Dataset/train.jsonl'
with open(load_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
print(len(data))

405573


In [4]:
C.Counter(d['source'] for d in data)

Counter({'Numina': 199036,
         'PromptCot': 131723,
         'AQuA': 59294,
         'GSM8K': 6979,
         'AoPS': 4750,
         'MATH': 3791})

In [5]:
data_main = [d for d in data if d['source'] == 'PromptCot']
print(len(data_main))

131723


In [6]:
informal_problem = data_main[0]['prompt'].split('<｜User｜>', 1)[-1]
assert informal_problem.endswith('<｜Assistant｜>')
informal_problem = informal_problem[:-len('<｜Assistant｜>')]
print(informal_problem)

Let $P$ be a point on a regular $n$-gon. A marker is placed on the vertex $P$ and a random process is used to select a vertex $Q$ uniformly from the remaining $n-1$ vertices. The marker is then moved from $P$ to $Q$ and from $Q$ to a vertex $R$ selected uniformly at random from the remaining $n-1$ vertices. This process is repeated $m$ times, and let $A_m$ be the angle determined by the marker at $P$ after $m$ steps. Let $k$ be a positive integer less than $n/2$ such that $A_k$ is in the angle region $\Delta n$ determined by $PQ$ and $PR$ and let $\lfloor x \rfloor$ denote the greatest integer less than or equal to $x$. Find the smallest positive integer $m$ such that $m \leq \lfloor 2n \lfloor \theta \rfloor \rfloor$, where $\theta = \frac{k\pi}{n}$.


In [7]:
informal_solution = data_main[0]['completion'].split('</think>')[-1]
informal_answer = M.parse(informal_solution)[-1]
print(informal_answer)

\lfloor 2n \lfloor \frac{k\pi}{n} \rfloor \rfloor


In [11]:
data_main_processed = []
for i, d in enumerate(data):
    if d['source'] == 'PromptCot':
        informal_problem = d['prompt'].split('<｜User｜>', 1)[-1]
        assert informal_problem.endswith('<｜Assistant｜>')
        informal_problem = informal_problem[:-len('<｜Assistant｜>')]
        # print(informal_problem)
        informal_solution = d['completion'].split('</think>')[-1]
        informal_answer = M.parse(informal_solution)
        try:
            assert len(informal_answer) == 2
            informal_answer = informal_answer[-1]
            # print(informal_answer)
            data_main_processed.append({
                'informal_problem': informal_problem,
                'informal_solution': informal_solution,
                'informal_answer': informal_answer,
                'source_path': load_path,
                'source_idx': i
            })
        except:
            pass
print(len(data), len(data_main_processed))

Timeout during parsing: 

Given the set \( S \) which is the power set of \( \{1, 2, 3, \ldots, 2^{18}\} \), we need to find the number of subsets of \( S \) that have exactly \( 2^8 \) elements.

1. The original set \( \{1, 2, 3, \ldots, 2^{18}\} \) has \( 2^{18} \) elements.
2. The power set \( S \) of this set has \( 2^{2^{18}} \) elements.
3. We need to find the number of subsets of \( S \) that have exactly \( 2^8 \) elements, which is 256.
4. The number of such subsets is given by the binomial coefficient \( \binom{2^{2^{18}}}{256} \).

Thus, the number of subsets of \( S \) that have exactly \( 2^8 \) elements is:

\[
\boxed{\dbinom{2^{2^{18}}}{256}}
\]


405573 131648


In [13]:
len(data_main)

131723

In [12]:
with open('/cache/data/fpg_informal_baselines/PromptCoT-DS.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

### PromptCoT-QwQ-Dataset

In [14]:
load_path = '/home/ma-user/local_cache/zhaoxlpku/PromptCoT-QwQ-Dataset/train.jsonl'
with open(load_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
print(len(data))

42887


In [15]:
C.Counter(d['source'] for d in data)

Counter({'PromptCot': 25238, 'Numina': 15421, 'AoPS': 2119, 'MATH': 109})

In [23]:
data_main_processed = []
for i, d in enumerate(data):
    if d['source'] == 'PromptCot':
        # informal_problem = d['prompt'].split('<｜User｜>', 1)[-1]
        # assert informal_problem.endswith('<｜Assistant｜>')
        # informal_problem = informal_problem[:-len('<｜Assistant｜>')]
        assert d['prompt'].startswith('<|im_start|>user\n') and d['prompt'].endswith('\nPlease reason step by step, and put your final answer within \\boxed{}.<|im_end|>\n<|im_start|>assistant')
        informal_problem = d['prompt'][len('<|im_start|>user\n'):-len('\nPlease reason step by step, and put your final answer within \\boxed{}.<|im_end|>\n<|im_start|>assistant')].strip()
        # print(informal_problem)
        informal_solution = d['completion'].split('</think>')[-1].strip()
        informal_answer = M.parse(informal_solution)
        try:
            assert len(informal_answer) == 2
            informal_answer = informal_answer[-1].strip()
            # print(informal_answer)
            data_main_processed.append({
                'informal_problem': informal_problem,
                'informal_solution': informal_solution,
                'informal_answer': informal_answer,
                'source_path': load_path,
                'source_idx': i
            })
        except:
            pass
print(len(data_main), len(data_main_processed))

131723 25238


In [24]:
data_main = [d for d in data if d['source'] == 'PromptCot']
print(len(data_main))

25238


In [25]:
informal_problem

'Let $\\triangle ABC$ be a triangle with $M$ as the midpoint of $\\overline{BC}$. Let $x$ be the length of $\\overline{AM}$ and $\\overline{CM}$. The median and mean of all possible values of $\\frac{BC}{AM}$ can be written in the form $\\frac{a}{b}$ and $\\frac{c}{d}$, where $a$, $b$, $c$, and $d$ are positive integers and $a$ and $c$ are relatively prime. Find $a+b+c+d$.'

In [26]:
with open('/cache/data/fpg_informal_baselines/PromptCoT-QwQ.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

### ScaleQuest-Math

In [79]:
load_path = '/home/ma-user/local_cache/dyyyyyyyy/ScaleQuest-Math/train.json'
with open(load_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
print(len(data))

1003467


In [80]:
data[0]

{'query': 'At what point do the graphs of $y=5x-26$ and $y=-\\frac{3}{4}x+19$ intersect?',
 'response': "To find the point of intersection of the graphs of the equations \\( y = 5x - 26 \\) and \\( y = -\\frac{3}{4}x + 19 \\), we need to find the values of \\( x \\) and \\( y \\) that satisfy both equations simultaneously. This means we need to set the right-hand sides of the equations equal to each other and solve for \\( x \\).\n\nStarting with the equations:\n\\[ y = 5x - 26 \\]\n\\[ y = -\\frac{3}{4}x + 19 \\]\n\nWe set the right-hand sides equal to each other:\n\\[ 5x - 26 = -\\frac{3}{4}x + 19 \\]\n\nTo eliminate the fraction, we can multiply every term by 4:\n\\[ 4(5x - 26) = 4\\left(-\\frac{3}{4}x + 19\\right) \\]\n\\[ 20x - 104 = -3x + 76 \\]\n\nNext, we want to get all the \\( x \\)-terms on one side of the equation and the constant terms on the other side. We do this by adding \\( 3x \\) to both sides:\n\\[ 20x + 3x - 104 = 76 \\]\n\\[ 23x - 104 = 76 \\]\n\nThen, we add 104 

In [81]:
print(data[0]['query'])

At what point do the graphs of $y=5x-26$ and $y=-\frac{3}{4}x+19$ intersect?


In [82]:
print(data[0]['response'])

To find the point of intersection of the graphs of the equations \( y = 5x - 26 \) and \( y = -\frac{3}{4}x + 19 \), we need to find the values of \( x \) and \( y \) that satisfy both equations simultaneously. This means we need to set the right-hand sides of the equations equal to each other and solve for \( x \).

Starting with the equations:
\[ y = 5x - 26 \]
\[ y = -\frac{3}{4}x + 19 \]

We set the right-hand sides equal to each other:
\[ 5x - 26 = -\frac{3}{4}x + 19 \]

To eliminate the fraction, we can multiply every term by 4:
\[ 4(5x - 26) = 4\left(-\frac{3}{4}x + 19\right) \]
\[ 20x - 104 = -3x + 76 \]

Next, we want to get all the \( x \)-terms on one side of the equation and the constant terms on the other side. We do this by adding \( 3x \) to both sides:
\[ 20x + 3x - 104 = 76 \]
\[ 23x - 104 = 76 \]

Then, we add 104 to both sides to isolate the term with \( x \):
\[ 23x = 180 \]

Now, we solve for \( x \) by dividing both sides by 23:
\[ x = \frac{180}{23} \]

With \( x

In [ ]:
data_main_processed = []
for i, d in enumerate(tqdm(data)):
    informal_answer = M.parse(d['response'])
    # assert len(informal_answer) == 2
    informal_answer = informal_answer[-1]
    assert isinstance(informal_answer, str)
    data_main_processed.append({
        'informal_problem': d['query'],
        'informal_solution': d['response'],
        'informal_answer': informal_answer,
        'source_path': load_path,
        'source_idx': i
    })
print(len(data_main_processed))

In [ ]:
# Use `/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/scalequest_processing.py` to process/
raise

In [ ]:
# def worker(args):
#     i, d, load_path = args
#     informal_answer = M.parse(d['response'])
#     # assert len(informal_answer) == 2
#     informal_answer = informal_answer[-1]
#     assert isinstance(informal_answer, str)
#     data_main_processed.append({
#         'informal_problem': d['query'],
#         'informal_solution': d['response'],
#         'informal_answer': informal_answer,
#         'source_path': load_path,
#         'source_idx': i
#     })
#     return data_main_processed

In [1]:
# import multiprocessing
# args_list = [(i, d, load_path) for i, d in enumerate(data)]

# with multiprocessing.Pool(processes=16) as pool:
#     data_main_processed = list(tqdm(
#         pool.imap(worker, args_list),
#         total=len(data),
#         desc="Processing data"
#     ))

In [37]:
with open('/cache/data/fpg_informal_baselines/ScaleQuest-Math.processed.jsonl', 'r') as f:
    data_main_processed = [json.loads(l) for l in f.readlines()]

In [39]:
C.Counter('exception' in d for d in data_main_processed), C.Counter('informal_answer' in d for d in data_main_processed)

(Counter({False: 1003466, True: 1}), Counter({True: 1003466, False: 1}))

In [40]:
for d in data_main_processed:
    if 'exception' in d:
        raise

RuntimeError: No active exception to reraise

In [41]:
print(d['exception'])

IndexError('list index out of range')


In [46]:
d['informal_answer'] = r'\binom{2013^{2012} - 1}{2012}'
d.pop('exception')

"IndexError('list index out of range')"

In [47]:
C.Counter('exception' in d for d in data_main_processed), C.Counter('informal_answer' in d for d in data_main_processed)

(Counter({False: 1003467}), Counter({True: 1003467}))

In [48]:
with open('/cache/data/fpg_informal_baselines/ScaleQuest-Math.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

### MUSTARDSauce

In [3]:
from xfinder.eval import Evaluator
from xfinder.modules import Extractor

extractor = Extractor(
    model_name='xFinder-qwen1505',
    inference_mode='local',
    model_path_or_url='/home/ma-user/local_cache/IAAR-Shanghai/xFinder-qwen1505',
    temperature=0,
    max_tokens=256,
)

answer_type = 'math'
answer_range = Evaluator.MATH_STANDARD_ANSWER_RANGE
if answer_type not in Evaluator.VALID_KEY_ANSWER_TYPES:
    raise ValueError(
        f"Invalid key_answer_type: {answer_type}. Must be one of {Evaluator.VALID_KEY_ANSWER_TYPES}")

/home/ma-user/anaconda3/envs/default/lib/python3.11/site-packages/torch_npu/utils/collect_env.py:58: UserWarning: Warning: The /usr/local/Ascend/nnae/latest owner does not match the current owner.
  warnings.warn(f"Warning: The {path} owner does not match the current owner.")
/home/ma-user/anaconda3/envs/default/lib/python3.11/site-packages/torch_npu/utils/collect_env.py:58: UserWarning: Warning: The /usr/local/Ascend/nnae/8.2.RC1/ascend_nnae_install.info owner does not match the current owner.
  warnings.warn(f"Warning: The {path} owner does not match the current owner.")
/home/ma-user/anaconda3/envs/default/lib/python3.11/site-packages/torch_npu/utils/collect_env.py:58: UserWarning: Warning: The /usr/local/Ascend/nnae/latest owner does not match the current owner.
  warnings.warn(f"Warning: The {path} owner does not match the current owner.")
/home/ma-user/anaconda3/envs/default/lib/python3.11/site-packages/torch_npu/utils/collect_env.py:58: UserWarning: Warning: The /usr/local/Ascen

In [4]:
load_path = '/cache/data/MUSTARDSauce/data/subset_filtered'
data = []
for p in os.listdir(load_path):
    with open(osp.join(load_path, p), 'r') as f:
        data.append(json.load(f) | {'file_name': p})
print(len(data))

5866


In [5]:
lean4_base = '/cache/data/MUSTARDSauce_lean4'

def transform_name(file_name: str) -> str:
    assert file_name.endswith('.json')
    chunks = file_name[:-len('.json')].replace(' ', '_').split('_')
    for (i, c) in enumerate(chunks):
        if c[0].islower():
            chunks[i] = c[0].upper() + c[1:]
    return ''.join(chunks) + '.lean'
# print(transform_name(d['file_name']))

transformed_file_names = {transform_name(d['file_name']) for d in data}
ported_file_names = set(os.listdir(lean4_base))
ported_file_names - transformed_file_names, transformed_file_names - ported_file_names

({'.lake',
  'Main.lean',
  'MiniF2F.lean',
  'Other.lean',
  'lake-manifest.json',
  'lakefile.lean',
  'lean-toolchain'},
 {'TpELEMK25th731Correct.lean',
  'TpHIGHK1High392Step.lean',
  'TpHIGHK2110165Correct.lean',
  'TpMIDDK1Algebra824Correct.lean',
  'TpMIDDK1Algebra953Correct.lean',
  'TpMIDDK2Algebra515Correct.lean',
  'WpELEMK13rd498Correct.lean',
  'WpMIDDK2Grade10043Correct.lean'})

In [6]:
C.Counter(
    osp.exists(osp.join(lean4_base, transform_name(d['file_name']))) for d in data
)

Counter({True: 5858, False: 8})

In [7]:
data[101]

{'problem_name': 'correct_by_msg__HIGH_theorem_proving_Calculus_1_542_round1',
 'informal_statement': 'Consider a random variable X that follows a probability distribution where X can take the values 1, 2, 3, and 4 with the probabilities 1/10, 2/10, 3/10, and 4/10 respectively. Calculate the expected value (mean) of the distribution.',
 'informal_proof': 'The expected value (mean) of a probability distribution is calculated by multiplying each outcome by its probability and then summing these values. \n\nIn our case, the expected value E(X) would be calculated as follows:\n\nE(X) = (1 * 1/10) + (2 * 2/10) + (3 * 3/10) + (4 * 4/10) \n     = 0.1 + 0.4 + 0.9 + 1.6 \n     = 3.0\n\nSo, the expected value of the distribution is 3.0.',
 'formal_proof': 'import data.real.basic\n\n-- make the definition noncomputable as mentioned in the error\nnoncomputable def X : (ℕ → ℝ) := λ x, if x = 1 then 1/10 else if x = 2 then 2/10 else if x = 3 then 3/10 else if x = 4 then 4/10 else 0\n\n-- proof of th

In [8]:
C.Counter(
    'In the following, you are given a' in d['informal_statement'] for d in data
)

Counter({False: 4703, True: 1163})

In [9]:
prompt_str = "In the following, you are given a ``Problem'', a pair of corresponding ``Informal proof'' and ``Formal proof in Lean 3'', along with error messages from a Lean prover corresponding to the ``Formal proof in Lean 3''. Now please carefully modify the ``Formal proof in Lean 3'' section so that it passes the Lean prover without error. You should write the modi fied complete proof in your response."
data_main_processed = []

for d in tqdm(data):
    if 'In the following' in d['informal_statement']:
        assert d['informal_statement'].startswith(prompt_str)
        data_main_processed.append({
            'informal_statement': d['informal_proof'],
            'formal_proof': d['formal_proof'],
            'source_path': load_path,
            'source_idx': d['problem_name'],
            'file_name': d['file_name'],
            'lean4_source_path': None if not osp.exists(osp.join(lean4_base, transform_name(d['file_name']))) else transform_name(d['file_name']),
        })
    else:
        data_main_processed.append({
            'informal_statement': d['informal_statement'],
            'informal_proof': d['informal_proof'],
            'informal_answer': extractor.generate_output(d['informal_statement'], d['informal_proof'], answer_range),
            'formal_proof': d['formal_proof'],
            'source_path': load_path,
            'source_idx': d['problem_name'],
            'file_name': d['file_name'],
            'lean4_source_path': None if not osp.exists(osp.join(lean4_base, transform_name(d['file_name']))) else transform_name(d['file_name'])
        })
print(len(data_main_processed))

100%|██████████| 5866/5866 [19:52<00:00,  4.92it/s]

5866


In [11]:
C.Counter(d.get('informal_answer') for d in data_main_processed)

Counter({None: 1163,
         '[No valid answer]': 760,
         '5': 104,
         '20': 94,
         '4': 89,
         '10': 86,
         '6': 83,
         '3': 78,
         '30': 76,
         '15': 75,
         '12': 72,
         '60': 63,
         '2': 62,
         '40': 59,
         '8': 59,
         '50': 58,
         '1': 55,
         '16': 51,
         '14': 50,
         '24': 50,
         '9': 49,
         '25': 42,
         '100': 40,
         '7': 39,
         '90': 38,
         '35': 37,
         '32': 33,
         '11': 33,
         '34': 29,
         '0': 28,
         '150': 27,
         '18': 26,
         '45': 26,
         '1/4': 25,
         '125': 24,
         '28': 23,
         '1/2': 23,
         '21': 23,
         '17': 21,
         '75': 21,
         '64': 21,
         '13': 21,
         '200': 21,
         '80': 20,
         '120': 19,
         '22': 19,
         '1/3': 19,
         '0.5': 19,
         '36': 18,
         '300': 18,
         '180': 17,
         '7

In [10]:
with open('/cache/data/fpg_informal_baselines/MUSTARDSauce.processed.jsonl', 'w') as f:
    for d in data_main_processed:
        f.write(json.dumps(d)+'\n')

In [91]:
code_pieces = []
for p in os.listdir(lean4_base):
    if osp.isdir(osp.join(lean4_base, p)) or any(t in p for t in ['lean-toolchain', 'lake']):
        continue
    with open(osp.join(lean4_base, p), 'r') as f:
        code = '\n'.join(['import Mathlib', 'import Aesop'] + [l.rstrip() for l in f.readlines() if not (l.startswith('#align_import ') or l.startswith('#align ') or l.startswith('import '))])
    # break
    with open(osp.join(lean4_base, p), 'w') as f:
        f.write(code)
    code_pieces.append((p, code))

In [92]:
import regex as re
C.Counter(
    len(re.findall('theorem ', remove_comments(c))) for (p, c) in code_pieces
)

Counter({0: 4439, 1: 1285, 2: 108, 3: 25, 4: 3, 5: 1})

In [93]:
import regex as re
C.Counter(
    len(re.findall('lemma ', remove_comments(c))) for (p, c) in code_pieces
)

Counter({0: 5860, 2: 1})

In [95]:
import regex as re
C.Counter(
    len(re.findall('example ', remove_comments(c))) for (p, c) in code_pieces
)

Counter({0: 2890, 1: 2708, 2: 212, 3: 44, 4: 6, 5: 1})

In [99]:
for (p, c) in code_pieces:
    if len(re.findall('example ', remove_comments(c))) == 4:
        raise

RuntimeError: No active exception to reraise

In [101]:
print(p)
print(c)

WpMIDDK2Algebra911Step.lean
import Mathlib
import Aesop


open Real

#check ((3 ^ 2 * 5) ^ 3 : ℝ) / (3 ^ 4 * 5 ^ 3 : ℝ)

-- check the type of our expression
-- Let's simplify step by step
-- Step 1: simplify the expression in the parentheses (3^2*5)
example : (3 ^ 2 * 5 : ℝ) = 45 := by norm_num

-- Step 2: apply the exponent 3 to the 45
example : (45 ^ 3 : ℝ) = 91125 := by norm_num

-- Step 3: simplify the denominator, 3^4 equals 81, and the 5^3 equals 125
example : (3 ^ 4 * 5 ^ 3 : ℝ) = 10125 := by norm_num

-- Final Step: we divide the numerator by the denominator
example : ((3 ^ 2 * 5) ^ 3 : ℝ) / (3 ^ 4 * 5 ^ 3 : ℝ) = 9 := by norm_num



In [ ]:
# for d in data_main_processed:
#     if d['lean4_source_path'] is None:
#         p = d['file_name'].replace('.json', '.lean')
#         print(f'rm {p}')

rm tp_MIDD_k2_Algebra_515_correct.lean
rm tp_HIGH_k2_1_10165_correct.lean
rm wp_MIDD_k2_grade_10043_correct.lean
rm wp_ELEM_k1_3rd_498_correct.lean
rm tp_MIDD_k1_Algebra_824_correct.lean
rm tp_MIDD_k1_Algebra_953_correct.lean
rm tp_ELEM_k2_5th_731_correct.lean
rm tp_HIGH_k1_High_392_step.lean


In [16]:
import_cnt = C.Counter()
open_cnt = C.Counter()
open_scoped_cnt = C.Counter()
option_cnt = C.Counter()

In [17]:
for p in code_pieces:
    for l in p.splitlines():
        if l.startswith('import '):
            import_cnt[l[len('import '):].strip()] += 1
        elif l.startswith('open scoped '):
            for t in l[len('open scoped '):].strip().split():
                open_scoped_cnt[t] += 1
        elif l.startswith('open '):
            for t in l[len('open '):].strip().split():
                open_cnt[t] += 1
        elif l.startswith('set_option '):
            option_cnt[l[len('set_option '):].strip()] += 1

In [18]:
import_cnt

Counter({'Mathlib': 5861, 'Aesop': 5861})

In [19]:
option_cnt

Counter()

In [20]:
open_cnt

Counter({'Nat': 611,
         'Real': 225,
         'Int': 141,
         'Matrix': 17,
         'Rat': 13,
         'Complex': 10,
         'Function': 9,
         'List': 5,
         'Set': 4,
         'intervalIntegral': 3,
         'Polynomial': 2,
         'Finset': 1,
         'Filter': 1,
         'Io': 1,
         'Mathlib.Vector': 1})

In [21]:
open_scoped_cnt

Counter({'Classical': 170, 'BigOperators': 6, 'Matrix': 2, 'Rat': 1})

In [102]:
for d in data:
    if 'In the following' in d['informal_statement']:
        raise
    else:
        pass

RuntimeError: No active exception to reraise

In [107]:
d_wp = [
    d for d in data if 'In the following' in d['informal_statement']
]
d_tp = [
    d for d in data if 'In the following' not in d['informal_statement']
]

In [131]:
d = random.choice(d_tp)
print(d['informal_statement'])
print(d['informal_proof'])

In a rectangle ABCD, AB = 5 units and BC = 3 units. What is the area of the rectangle ABCD?
Area of a rectangle is computed by multiplying the length of its sides. In this case, the sides of the rectangle are AB = 5 units and BC = 3 units. Hence, the area of rectangle ABCD is `5 units * 3 units = 15 square units`.


In [139]:
d = random.choice(d_wp)
# print(d['informal_statement'])
print(d['informal_proof'])

Sarah has 35 apples in her basket. She gives 12 apples to her friend John. Later, her mom gives her 20 more apples. Estimate how many apples Sarah has now.
